### Código para limpiar sensores 

In [125]:
import pandas as pd

df = pd.read_csv('Sensores_limpio_corregido_agrupado_ok.csv')  

df_sin_bateria = df[~df['entity_id'].str.contains('_battery', na=False)]

# Eliminamos filas que contienen un sensor nuevo que solo hace un registro 3 veces
df_sin_bateria = df_sin_bateria.drop(df[df['entity_id'].isin(['sensor.sensor_temperatura_3_nuevo_4_humidity','sensor.sensor_temperatura_3_nuevo_4_pressure','sensor.sensor_temperatura_3_nuevo_4_temperature'])].index)


sensores_ventanas_y_puerta = df_sin_bateria[
    (df_sin_bateria['entity_id'].str.contains('sensor.sensor_ventana_', na=False)) | 
    (df_sin_bateria['entity_id'] == 'sensor.sensor_puerta_1_device_temperature')
]

sensores_temperatura = df_sin_bateria[df_sin_bateria['entity_id'].str.contains('sensor.sensor_temperatura_', na=False)]

sensores_ventanas_y_puerta.to_csv('Sensores_ventanas_y_puerta_agrupados.csv', index=False)
sensores_temperatura.to_csv('Sensores_temperatura_clase_agrupados.csv', index=False)

sensores_temperatura["entity_id"].unique()


array(['sensor.sensor_temperatura_1_humidity',
       'sensor.sensor_temperatura_1_pressure',
       'sensor.sensor_temperatura_1_temperature',
       'sensor.sensor_temperatura_2_humidity',
       'sensor.sensor_temperatura_2_pressure',
       'sensor.sensor_temperatura_2_temperature',
       'sensor.sensor_temperatura_3_humidity',
       'sensor.sensor_temperatura_3_pressure',
       'sensor.sensor_temperatura_3_temperature',
       'sensor.sensor_temperatura_4_humidity',
       'sensor.sensor_temperatura_4_pressure',
       'sensor.sensor_temperatura_4_temperature'], dtype=object)

### Código para limpiar csv de sol, quedando solo las variables de interés para predecir la temperatura del aula

In [126]:
df_sol_sin_agrupar = pd.read_csv('CSV_datos_Sol_limpio.csv', parse_dates=['time'])  

df_sol_sin_agrupar = df_sol_sin_agrupar[["time", "azimuth", "elevation"]]

df_sol_sin_agrupar.to_csv('Datos_sol_variables_importantes.csv', index=False)


### Agrupamos el csv de sol para que sea cada media hora

In [127]:
# Intentar convertir la columna 'time' asegurando compatibilidad con formatos variables
df_sol_sin_agrupar['time'] = pd.to_datetime(df_sol_sin_agrupar['time'], format='ISO8601', errors='coerce')

# Remover la zona horaria si existe
df_sol_sin_agrupar['time'] = df_sol_sin_agrupar['time'].dt.tz_localize(None)

# Convertir las columnas a numérico, ignorando errores
df_sol_sin_agrupar['azimuth'] = pd.to_numeric(df_sol_sin_agrupar['azimuth'], errors='coerce')
df_sol_sin_agrupar['elevation'] = pd.to_numeric(df_sol_sin_agrupar['elevation'], errors='coerce')

# Agrupar por intervalos de 30 minutos y calcular la media de 'azimuth' y 'elevation'
df_sol_agrupado = df_sol_sin_agrupar.groupby(pd.Grouper(key='time', freq='60T')).agg(
    azimuth_mean=('azimuth', 'mean'),
    elevation_mean=('elevation', 'mean')
).reset_index()

# Redondear 'valor_medio' a 2 decimales
df_sol_agrupado['azimuth_mean'] = df_sol_agrupado['azimuth_mean'].round(2)
df_sol_agrupado['elevation_mean'] = df_sol_agrupado['elevation_mean'].round(2)



df_sol_agrupado.to_csv("./datos_definitivos_sol_agrupados.csv", index=False)
df_sol_agrupado


/tmp/ipykernel_941/846145110.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_sol_agrupado = df_sol_sin_agrupar.groupby(pd.Grouper(key='time', freq='60T')).agg(


,time,azimuth_mean,elevation_mean
0,2024-12-10 18:00:00,258.46,-23.01
1,2024-12-10 19:00:00,264.24,-30.60
2,2024-12-10 20:00:00,273.64,-42.09
3,2024-12-10 21:00:00,285.41,-53.48
4,2024-12-10 22:00:00,303.03,-64.02
...,...,...,...
2516,2025-03-25 14:00:00,NaN,NaN
2517,2025-03-25 15:00:00,NaN,NaN
2518,2025-03-25 16:00:00,NaN,NaN
2519,2025-03-25 17:00:00,NaN,NaN


### Código para limpiar csv de forecast
Nos hemos dado cuenta que los datos de la entidad sensor mislata y la entidad forecast weather parece ser los mismos, por lo que solo usaremos los datos de mislata

In [128]:
import json


df_raw = pd.read_csv("./entidad_weather_forecast.csv", sep=';', parse_dates=['time'])
#df_raw = pd.read_csv("./entidad_sensor_mislata.csv", sep=';', parse_dates=['time'])

# Si tiene columna 'attributes', expandirla
if 'attributes' in df_raw.columns:
    df_raw['attributes'] = df_raw['attributes'].apply(json.loads)
    atributos_expandidos = pd.json_normalize(df_raw['attributes'])
    df_limpio = pd.concat([df_raw.drop(columns=['attributes']), atributos_expandidos], axis=1)
else:
    df_limpio = df_raw


# Guardar el archivo limpio como CSV
df_limpio.to_csv("./forecast_limpio.csv", index=False)


df_silver = df_limpio[df_limpio['entity_id'] != "weather.forecast_casa"]
df_silver = df_silver[df_silver['state'] != 'unavailable']



print("Columnas originales",df_silver.columns)

# Nos quedamos con las columnas que nos interesan
df_silver = df_silver[["time", "temperature", "cloud_coverage"]]
df_silver['time'] = pd.to_datetime(df_silver['time']).dt.strftime('%Y-%m-%d %H:%M:%S')

df_silver.to_csv('pronostico_exterior.csv', index=False)
df_silver


Columnas originales Index(['time', 'entity_id', 'state', 'humidity', 'pressure', 'dew_point',
       'wind_speed', 'attribution', 'temperature', 'wind_bearing',
       'friendly_name', 'pressure_unit', 'cloud_coverage', 'visibility_unit',
       'wind_speed_unit', 'temperature_unit', 'precipitation_unit',
       'supported_features', 'restored'],
      dtype='object')


,time,temperature,cloud_coverage
166,2024-12-19 02:08:22,13.8,100.0
178,2024-12-18 17:03:23,16.3,34.4
179,2024-12-18 19:05:23,15.0,53.9
180,2024-12-18 18:04:23,15.4,55.5
181,2024-12-18 19:29:21,15.0,53.9
...,...,...,...
2477,2025-03-24 07:12:21,8.9,81.2
2478,2025-03-24 10:21:21,15.3,1.6
2479,2025-03-24 14:33:51,18.1,49.2
2480,2025-03-24 18:45:52,13.6,99.2


In [129]:
import json
import pandas as pd


df_raw = pd.read_csv("./entidad_sensor_mislata.csv", sep=';', parse_dates=['time'])
print("Columnas originales",df_raw.columns)
'''

# Lista de columnas que quieres en la salida
desired_columns = [
    'sensor.sensor_temperatura_1_humidity',
    'sensor.sensor_temperatura_1_pressure',
    'sensor.sensor_temperatura_1_temperature',
    'sensor.sensor_temperatura_2_humidity',
    'sensor.sensor_temperatura_2_pressure',
    'sensor.sensor_temperatura_2_temperature',
    'sensor.sensor_temperatura_3_humidity',
    'sensor.sensor_temperatura_3_pressure',
    'sensor.sensor_temperatura_3_temperature',
    'sensor.sensor_temperatura_4_humidity',
    'sensor.sensor_temperatura_4_pressure',
    'sensor.sensor_temperatura_4_temperature'
]

# 1. Leer el CSV
df = pd.read_csv("Sensores_temperatura_clase_agrupados.csv", parse_dates=["time"])

# 2. Asegurar que 'time' es índice
df.set_index("time", inplace=True)

# 3. Pivotar la tabla para tener cada entity_id como columna
df_pivot = df.pivot(columns="entity_id", values="valor_medio")

# 4. Crear un rango de fechas cada 30 minutos (media hora) desde la min a la max
all_times = pd.date_range(start=df_pivot.index.min(), end=df_pivot.index.max(), freq="60T")

# 5. Reindexar y rellenar valores ausentes con el último valor disponible
df_pivot = df_pivot.reindex(all_times)
df_pivot = df_pivot.ffill()

# 6. Reindexar para quedarnos sólo con las columnas deseadas
df_pivot = df_pivot.reindex(columns=desired_columns)

# Opcional: restaurar el índice como columna si prefieres 'time' en lugar de índice
df_pivot.reset_index(inplace=True)
df_pivot.rename(columns={"index": "time"}, inplace=True)

# Ya tienes el dataframe final con las columnas deseadas y filas cada media hora
print(df_pivot)

df_pivot.to_csv('sensores_clase_temperatura_agrupados.csv', index=False)
'''

Columnas originales Index(['time', 'entity_id', 'state', 'attributes'], dtype='object')


'\n\n# Lista de columnas que quieres en la salida\ndesired_columns = [\n    \'sensor.sensor_temperatura_1_humidity\',\n    \'sensor.sensor_temperatura_1_pressure\',\n    \'sensor.sensor_temperatura_1_temperature\',\n    \'sensor.sensor_temperatura_2_humidity\',\n    \'sensor.sensor_temperatura_2_pressure\',\n    \'sensor.sensor_temperatura_2_temperature\',\n    \'sensor.sensor_temperatura_3_humidity\',\n    \'sensor.sensor_temperatura_3_pressure\',\n    \'sensor.sensor_temperatura_3_temperature\',\n    \'sensor.sensor_temperatura_4_humidity\',\n    \'sensor.sensor_temperatura_4_pressure\',\n    \'sensor.sensor_temperatura_4_temperature\'\n]\n\n# 1. Leer el CSV\ndf = pd.read_csv("Sensores_temperatura_clase_agrupados.csv", parse_dates=["time"])\n\n# 2. Asegurar que \'time\' es índice\ndf.set_index("time", inplace=True)\n\n# 3. Pivotar la tabla para tener cada entity_id como columna\ndf_pivot = df.pivot(columns="entity_id", values="valor_medio")\n\n# 4. Crear un rango de fechas cada 30 mi

### Agrupamos el csv de pronostico para que sea cada hora

In [130]:

# Intentar convertir la columna 'time' asegurando compatibilidad con formatos variables
df_silver['time'] = pd.to_datetime(df_silver['time'], format='ISO8601', errors='coerce')

# Remover la zona horaria si existe
df_silver['time'] = df_silver['time'].dt.tz_localize(None)

# Convertir las columnas a numérico, ignorando errores
df_silver['temperature'] = pd.to_numeric(df_silver['temperature'], errors='coerce')
df_silver['cloud_coverage'] = pd.to_numeric(df_silver['cloud_coverage'], errors='coerce')

# Agrupar por intervalos de 30 minutos y calcular la media de 'temperature' y 'cloud_coverage'
df_forecast_agrupado = df_silver.groupby(pd.Grouper(key='time', freq='60T')).agg(
    temperature_mean=('temperature', 'mean'),
    cloud_coverage_mean=('cloud_coverage', 'mean')
).reset_index()

# Redondear 'valor_medio' a 2 decimales
df_forecast_agrupado['temperature_mean'] = df_forecast_agrupado['temperature_mean'].round(2)
df_forecast_agrupado['cloud_coverage_mean'] = df_forecast_agrupado['cloud_coverage_mean'].round(2)



df_forecast_agrupado.to_csv("./datos_definitivos_pronostico_agrupado.csv", index=False)
df_forecast_agrupado


/tmp/ipykernel_941/11498688.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_forecast_agrupado = df_silver.groupby(pd.Grouper(key='time', freq='60T')).agg(


,time,temperature_mean,cloud_coverage_mean
0,2024-12-18 17:00:00,16.3,34.4
1,2024-12-18 18:00:00,15.4,55.5
2,2024-12-18 19:00:00,15.0,53.9
3,2024-12-18 20:00:00,14.7,1.6
4,2024-12-18 21:00:00,14.4,8.6
...,...,...,...
2305,2025-03-24 18:00:00,13.6,99.2
2306,2025-03-24 19:00:00,12.6,41.4
2307,2025-03-24 20:00:00,11.9,53.9
2308,2025-03-24 21:00:00,11.1,36.7


### Pivotar de filas individuales x entity, a agrupado x hora con todos los sensores

In [131]:
import pandas as pd

# Lista de columnas que quieres en la salida
desired_columns = [
    'sensor.sensor_temperatura_1_humidity',
    'sensor.sensor_temperatura_1_pressure',
    'sensor.sensor_temperatura_1_temperature',
    'sensor.sensor_temperatura_2_humidity',
    'sensor.sensor_temperatura_2_pressure',
    'sensor.sensor_temperatura_2_temperature',
    'sensor.sensor_temperatura_3_humidity',
    'sensor.sensor_temperatura_3_pressure',
    'sensor.sensor_temperatura_3_temperature',
    'sensor.sensor_temperatura_4_humidity',
    'sensor.sensor_temperatura_4_pressure',
    'sensor.sensor_temperatura_4_temperature'
]

# 1. Leer el CSV
df = pd.read_csv("Sensores_temperatura_clase_agrupados.csv", parse_dates=["time"])

# 2. Asegurar que 'time' es índice
df.set_index("time", inplace=True)

# 3. Pivotar la tabla para tener cada entity_id como columna
df_pivot = df.pivot(columns="entity_id", values="valor_medio")

# 4. Crear un rango de fechas cada 30 minutos (media hora) desde la min a la max
all_times = pd.date_range(start=df_pivot.index.min(), end=df_pivot.index.max(), freq="60T")

# 5. Reindexar y rellenar valores ausentes con el último valor disponible
df_pivot = df_pivot.reindex(all_times)
df_pivot = df_pivot.ffill()

# 6. Reindexar para quedarnos sólo con las columnas deseadas
df_pivot = df_pivot.reindex(columns=desired_columns)

# Opcional: restaurar el índice como columna si prefieres 'time' en lugar de índice
df_pivot.reset_index(inplace=True)
df_pivot.rename(columns={"index": "time"}, inplace=True)

# Ya tienes el dataframe final con las columnas deseadas y filas cada media hora
print(df_pivot)

df_pivot.to_csv('sensores_clase_temperatura_agrupados.csv', index=False)

entity_id                time  sensor.sensor_temperatura_1_humidity  \
0         2024-12-10 18:00:00                                 46.44   
1         2024-12-10 19:00:00                                 48.02   
2         2024-12-10 20:00:00                                 47.70   
3         2024-12-10 21:00:00                                 46.30   
4         2024-12-10 22:00:00                                 45.78   
...                       ...                                   ...   
2496      2025-03-24 18:00:00                                 47.17   
2497      2025-03-24 19:00:00                                 47.73   
2498      2025-03-24 20:00:00                                 48.80   
2499      2025-03-24 21:00:00                                 49.68   
2500      2025-03-24 22:00:00                                 50.02   

entity_id  sensor.sensor_temperatura_1_pressure  \
0                                        1013.5   
1                                        1012

/tmp/ipykernel_941/185480371.py:29: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  all_times = pd.date_range(start=df_pivot.index.min(), end=df_pivot.index.max(), freq="60T")


### Con los datos de contacto en ventanas y puerta, agrupamos por media hora y segundos abiertos en ese lapso de tiempo

In [132]:
import pandas as pd


df = pd.read_csv('Sensores_limpio_corregido.csv')  

df_contact = df[df['entity_id'].str.contains('contact', na=False)]

'''
Nos hace falta dado el df_contact tenemos que juntar la ultima tabla creada en gold, para que si la puerta se ha abierto en el plazo de los 30 minutos.
Se sumen los segundos y se cree una nueva columna con el tiempo total que ha estado abierta cada ventana/puerta.
'''

# Aseguramos que 'time' es datetime y ordenamos por sensor y tiempo
df_contact['time'] = pd.to_datetime(df_contact['time'])
df_contact = df_contact.sort_values(['friendly_name', 'time'])

# Convertimos 'state' a valores binarios: 1 para 'on' y 0 para 'off'
df_contact['state'] = df_contact['state'].map({'on': 1, 'off': 0})

# Lista para ir acumulando los registros distribuidos por intervalo de 30 minutos
records = []

# Procesamos cada sensor por separado
for sensor, group in df_contact.groupby('friendly_name'):
    group = group.sort_values('time').reset_index(drop=True)
    
    # Recorremos cada fila. Solo nos interesa cuando el sensor está en "on"
    for i, row in group.iterrows():
        if row['state'] == 1:
            start_time = row['time']
            # Se toma la siguiente fila para definir el final del período "on"
            if i < len(group) - 1:
                end_time = group.loc[i + 1, 'time']
            else:
                # Si no hay siguiente registro, se ignora este período
                continue

            # Si por alguna razón end_time es anterior a start_time, se omite
            if end_time <= start_time:
                continue

            # Determinamos el inicio del primer bucket de 30 minutos (se trunca a la hora)
            bucket_start = start_time.floor('60T')
            
            # Iteramos mientras el bucket actual se traslape con el período "on"
            while bucket_start < end_time:
                bucket_end = bucket_start + pd.Timedelta(minutes=60)
                # Calculamos la intersección entre el período "on" y el intervalo actual
                overlap_start = max(start_time, bucket_start)
                overlap_end = min(end_time, bucket_end)
                overlap_seconds = (overlap_end - overlap_start).total_seconds()
                if overlap_seconds > 0:
                    records.append({
                        'friendly_name': sensor,
                        'time': bucket_start,  # Usamos el inicio del bucket como clave
                        'duration': overlap_seconds
                    })
                bucket_start = bucket_end

# Creamos un DataFrame a partir de los registros
df_records = pd.DataFrame(records)

# Agrupamos por sensor y por cada bucket (intervalo de 30 minutos) sumando las duraciones
df_aggregated = df_records.groupby(['friendly_name', 'time']).agg(segundos_abierta=('duration', 'sum')).reset_index()

# Pivotamos para que cada sensor sea una columna, manteniendo 'time' como índice
df_pivot_contacto = df_aggregated.pivot(index='time', columns='friendly_name', values='segundos_abierta').fillna(0).reset_index()
df_pivot_contacto


# Hacemos suma de ventanas
columnas_ventanas_arriba = [
    " sensor_ventana_2 Puerta",
    "sensor_ventana_4 Puerta",
    "sensor_ventana_6 Puerta",
    "sensor_ventana_8 Puerta",
    "sensor_ventana_12 Puerta"
]

df_pivot_contacto["suma_ventanas_arriba"] = df_pivot_contacto[columnas_ventanas_arriba].sum(axis=1)

columnas_ventanas_abajo = [
    "sensor_ventana_1 Puerta",
    "sensor_ventana_3 Puerta",
    "sensor_ventana_5 Puerta",
    "sensor_ventana_7 Puerta",
    "sensor_ventana_9 Puerta",
    "sensor_ventana_11 Puerta"
]

df_pivot_contacto["suma_ventanas_abajo"] = df_pivot_contacto[columnas_ventanas_arriba].sum(axis=1)

df_pivot_contacto



/tmp/ipykernel_941/1110925025.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_contact['time'] = pd.to_datetime(df_contact['time'])
/tmp/ipykernel_941/1110925025.py:43: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  bucket_start = start_time.floor('60T')
/tmp/ipykernel_941/1110925025.py:43: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  bucket_start = start_time.floor('60T')
/tmp/ipykernel_941/1110925025.py:43: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  bucket_start = start_time.floor('60T')
/tmp/ipykernel_941/1110925025.py:43: FutureWarning: 'T' is deprecated and will be r

friendly_name,time,sensor_ventana_2 Puerta,sensor_puerta_1 Puerta,sensor_ventana_1 Puerta,sensor_ventana_11 Puerta,sensor_ventana_12 Puerta,sensor_ventana_3 Puerta,sensor_ventana_4 Puerta,sensor_ventana_5 Puerta,sensor_ventana_6 Puerta,sensor_ventana_7 Puerta,sensor_ventana_8 Puerta,sensor_ventana_9 Puerta,suma_ventanas_arriba,suma_ventanas_abajo
0,2025-01-17 13:00:00,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2025-01-17 14:00:00,83.0,1338.0,101.0,4.0,4.0,0.0,0.0,4.0,0.0,2303.0,3.0,3.0,90.0,90.0
2,2025-01-17 15:00:00,0.0,2404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3600.0,0.0,0.0,0.0,0.0
3,2025-01-17 16:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3457.0,0.0,0.0,0.0,0.0
4,2025-01-17 19:00:00,0.0,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,2025-03-24 15:00:00,0.0,3534.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1317,2025-03-24 16:00:00,0.0,3600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1318,2025-03-24 17:00:00,0.0,3600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1319,2025-03-24 18:00:00,0.0,3600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Merge del csv de sensores de temperatura y contacto

In [133]:


# Fusionar df_pivot con el contacto, dejando solo las columnas que nos interesan para reducir dimensionalidad, en base a 'time'
df_final = df_pivot.merge(df_pivot_contacto[["time", "sensor_puerta_1 Puerta", "suma_ventanas_arriba", "suma_ventanas_abajo"]], on='time', how='left')

cols_a_rellenar = ["sensor_puerta_1 Puerta", "suma_ventanas_arriba", "suma_ventanas_abajo"]

# Rellenar los valores NaN con 0, porque realmente son 0
df_final[cols_a_rellenar] = df_final[cols_a_rellenar].fillna(0)

df_final.to_csv('final_modelo_temperatura_y_binario_aula.csv', index=False)



### Merge  del csv de sensores de temperatura y contacto con informacion del sol

In [134]:
# Fusionar df_pivot con otro DataFrame en base a 'time'
df_final = df_final.merge(df_sol_agrupado, on='time', how='left')

df_final.to_csv('final_sensores_y_sol.csv', index=False)


### Merge del csv sensores y sol + pronostico de tiempo
 Hay varios datos vacios entre horas

In [135]:
# Fusionar df_pivot con otro DataFrame en base a 'time'
df_final = df_final.merge(df_forecast_agrupado, on='time', how='left')

df_final.to_csv('final_sensores_y_clima.csv', index=False)

### Merge de la y_true (temperatura sensor calefacción)

In [136]:
df_y_true = pd.read_csv('Histórico_temperaturas.csv', parse_dates=['time'])

df_final = df_final.merge(df_y_true, on='time', how='left')

df_final.to_csv('final_silver.csv', index=False)